In [1]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import sqlite3
key_fd = open('./과제data/kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [2]:
key_fd = open('./과제data/seoulapikey.txt', mode='r')
seoul_key = key_fd.read(100)
key_fd.close()

In [3]:
seoul_url = 'http://openapi.seoul.go.kr:8088/'
url = f'{seoul_url}{seoul_key}/xml/Corona19Status/1/1000'

In [4]:
url

'http://openapi.seoul.go.kr:8088/694f57486f686a6b33387443706a41/xml/Corona19Status/1/1000'

In [5]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('MESSAGE').get_text()

'정상 처리되었습니다'

In [6]:
total_count = int(soup.find('list_total_count').get_text())

In [7]:
total_count/1000

45.358

In [8]:
count = math.ceil(total_count/1000)
count

46

In [9]:
rows = soup.find_all('row')

In [10]:
rows[0].find('CORONA19_DATE').text

'2021-06-06'

In [11]:
rows[0].find('CORONA19_AREA').text

'타시도'

In [12]:
rows[0].find('CORONA19_ID').text

'45358'

In [13]:
rows[0].find('CORONA19_CONTACT_HISTORY').text

'감염경로 조사중'

In [14]:
id_list, area_list, date_list,histroy_list = [],[],[],[]
a_url = f'{seoul_url}{seoul_key}/xml/Corona19Status'
x = 1
for i in range(count):  
    url = f'{a_url}/{x}/{x+999}'
    x += 1000
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')
    rows = soup.find_all('row')
    for row in rows:
        id_list.append(row.find('CORONA19_ID').text)
        area_list.append(row.find('CORONA19_AREA').text)
        date_list.append(row.find('CORONA19_DATE').text)
        histroy_list.append(row.find('CORONA19_CONTACT_HISTORY').text)

In [15]:
df = pd.DataFrame({
    '연번' : id_list,
    '확진일' : date_list,
    '지역' : area_list,
    '접촉력' : histroy_list
})

In [16]:
len(histroy_list)

45358

In [17]:
df['지역'] = df['지역'].str.replace(' ','')

In [18]:
df['지역'][df['지역'] == '중로구']

Series([], Name: 지역, dtype: object)

In [19]:
df['지역'][df['지역'] == '중로구'] = '종로구'
df['지역'][df['지역'] == '앙쳔구'] = '양천구'

In [20]:
df['지역'][df['지역'] == '앙쳔구']

Series([], Name: 지역, dtype: object)

In [21]:
df['확진일']

0        2021-06-06
1        2021-06-06
2        2021-06-06
3        2021-06-06
4        2021-06-06
            ...    
45353    2020-01-31
45354    2020-01-30
45355    2020-01-30
45356    2020-01-30
45357    2020-01-24
Name: 확진일, Length: 45358, dtype: object

In [22]:
df['확진일'] = pd.to_datetime(df['확진일'])

In [23]:
df['확진일'] = df['확진일'].astype(str)

In [24]:
df = df.sort_values(by='확진일')

In [25]:
df.reset_index(drop=True,inplace=True)

In [26]:
df.tail()

,연번,확진일,지역,접촉력
45353,45255,2021-06-06,강남구,감염경로 조사중
45354,45256,2021-06-06,송파구,기타 확진자 접촉
45355,45257,2021-06-06,용산구,기타 확진자 접촉
45356,45279,2021-06-06,송파구,송파구 소재 시장 관련(\'21.5.)
45357,45358,2021-06-06,타시도,감염경로 조사중


In [27]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [28]:
sql_table = '''
    create table '서울시 확진자 현황'(
        '연번' text not null primary key, '확진일' text not null,
        '지역' text not null, 접촉력 text not null
    )
'''

In [29]:
cur.execute(sql_table)
conn.commit()

In [30]:
sql_insert = 'insert into "서울시 확진자 현황" values(?,?,?,?)'

In [31]:
for i in df.index:
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [32]:
conn.close()

In [33]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()
cur.execute(f'select * from "서울시 확진자 현황"')
rows = cur.fetchall()
conn.close()

In [34]:
df = pd.DataFrame(rows, columns=['연번','확진일','지역','접촉력'])

In [35]:
df

,연번,확진일,지역,접촉력
0,1,2020-01-24,강서구,해외유입
1,4,2020-01-30,마포구,해외유입
2,2,2020-01-30,중랑구,해외유입
3,3,2020-01-30,종로구,종로구 집단발병
4,5,2020-01-31,성북구,기타 확진자 접촉
...,...,...,...,...
45353,45255,2021-06-06,강남구,감염경로 조사중
45354,45256,2021-06-06,송파구,기타 확진자 접촉
45355,45257,2021-06-06,용산구,기타 확진자 접촉
45356,45279,2021-06-06,송파구,송파구 소재 시장 관련(\'21.5.)


In [36]:
sam = df[df['지역'] == '강서구'].pivot_table('연번','접촉력',aggfunc="count")

In [37]:
sam.sort_values(by='연번', ascending=False).head(10)

,연번
접촉력,
기타 확진자 접촉,734
감염경로 조사중,479
강서구 교회 관련,170
강서구 댄스교습관련 시설,163
타시도 확진자 접촉,114
해외유입,40
강서구 요양시설 관련,24
강서구 댄스교습 연관 병원 관련,20
8.15서울도심집회,18


In [38]:
list(sam.sort_values(by='연번', ascending=False).head(10).index)

['기타 확진자 접촉',
 '감염경로 조사중',
 '강서구 교회 관련',
 '강서구 댄스교습관련 시설',
 '타시도 확진자 접촉',
 '해외유입',
 '강서구 요양시설 관련',
 '강서구 댄스교습 연관 병원 관련',
 '8.15서울도심집회',
 '병원 및 요양시설']

In [39]:
sam.sort_values(by='연번', ascending=False).head(10)['연번']

접촉력
기타 확진자 접촉            734
감염경로 조사중             479
강서구 교회 관련            170
강서구 댄스교습관련 시설        163
타시도 확진자 접촉           114
해외유입                  40
강서구 요양시설 관련           24
강서구 댄스교습 연관 병원 관련     20
8.15서울도심집회            18
병원 및 요양시설             16
Name: 연번, dtype: int64

In [40]:
a_d = df.pivot_table('연번', '지역', aggfunc='count')

In [41]:
a_d.rename({'연번':'확진자수'},axis=1, inplace=True)

In [42]:
a_d

,확진자수
지역,
강남구,2588
강동구,1825
강북구,1310
강서구,2222
관악구,2081
광진구,1486
구로구,1512
금천구,754
기타,2105


In [43]:
d_day = df[df['지역'] == '강서구'].pivot_table('연번','확진일',aggfunc="count")

In [44]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)

In [45]:
list(ten_day.index)[::-1]


['2021-05-28',
 '2021-05-29',
 '2021-05-30',
 '2021-05-31',
 '2021-06-01',
 '2021-06-02',
 '2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06']

'2020-12-28'

In [46]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)
ten_date = list(ten_day.index)
ten_val = list(ten_day['연번'].values)